## 1.. Lets read data !

In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [16]:
df = pd.read_csv('/home/mis/Downloads/Hotel Bookings/hotel_bookings.csv')

filter1 = (df['children']==0) & (df['adults']==0) & (df['babies']==0)
## Adults,babies & children cant be zero at a same time bcz booking couldn't be possible if these 3 attributes are 0 ..
df2 = df[~filter1]
data= df2.drop_duplicates()
data[['lead_time' , 'total_of_special_requests' , 'adr']].describe().T
## getting (mean, median , std , percentile) of above features
data = data.reset_index(drop=True)

dict_month = {'July':7, 'August':8, 'September':9, 'October':10, 'November':11, 'December':12,
       'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6}

data['arrival_date_month_index'] = data['arrival_date_month'].map(dict_month)
data['arrival_date'] = data['arrival_date_year'].astype(str) + '-' + data['arrival_date_month_index'].astype(str) + '-' + data['arrival_date_day_of_month'].astype(str)
data['arrival_date'] = pd.to_datetime(data['arrival_date']).dt.date

In [17]:
data = data[data['arrival_date'].astype(str)>='2016-01-01']

In [18]:
data.to_csv("/home/mis/Downloads/Hotel Bookings/proccessed_input_hotel_bookings.csv",index=False)

In [ ]:
'''
print quantile values ..

'''

for col in ['lead_time' , 'total_of_special_requests' , 'adr']:
    print('feature name : {}'.format(col))

    for i in range(90,101,1):
        quantile_value = np.quantile(data[col] , q=i/100)
        print('{}th quantile value is {}'.format(i , quantile_value))
    print('\n')
    
'''
"adr" feature seems to have Outlier as 99th percentile value
is 261 but 100th percentile(max value) is 5400 ..
'''

In [ ]:
df.head()

In [ ]:
not_cancelled = data[data['is_canceled']==0]
country_wise_data = not_cancelled['country'].value_counts().reset_index()
country_wise_data.columns = ['country' , 'No of guests']


!pip install chart-studio
!pip install plotly

### establishing the entire set-up of Plotly..

import chart_studio.plotly as py
## chart_studio provides a web-service for hosting graphs!

import plotly.graph_objs as go
import plotly.express as px

from plotly.offline import download_plotlyjs , init_notebook_mode , plot , iplot

## iplot() when working in a Jupyter Notebook to
## display the plot in the Ipython notebook.

init_notebook_mode(connected=True)


# show on map

map_guest = px.choropleth(data_frame = country_wise_data ,
              locations= country_wise_data['country'] ,
              color=country_wise_data['No of guests'] ,
              hover_name=country_wise_data['country'] ,
              title= "Home country of Guests"

             )

map_guest.show()

'''Most guests are from Portugal and other countries in Europe'''


In [ ]:
category_pivot = pd.crosstab(index = data['reserved_room_type'] , columns=data['assigned_room_type'] , margins=True  )

'''
lets find meaningful insight from this :
for A category room , 56436 folks have reserved "A" & 45850 folks get assigned_room as "A".. & rest are unable to get !
for B category room , 996 folks have reserved "B" &  872 folks get assigned_room as "B".. & rest are unable to get !
'''

category_pivot

In [ ]:
## we will say just normalize over row , hence we need to pass normalize = 'index'
pivot_normalize = pd.crosstab(index = data['reserved_room_type'] , columns=data['assigned_room_type'] , margins=True ,normalize='index' ).round(2)*100

'''
Q.. Is any difference between assigned and reserved room type ?
Ans : Yes
'''
pivot_normalize

In [ ]:
data.shape

In [ ]:
# pie plot
fig = px.pie(data ,
      values = data['market_segment'].value_counts().values ,
      names = data['market_segment'].value_counts().index)
fig.show()